In [18]:
ASIN = input('Give ASIN code:')
#B08PCFVC58, B09KTX5FKM
url = 'https://www.amazon.co.uk/dp/' + ASIN

#https://www.amazon.co.uk/gp/bestsellers/?ref_=nav_cs_bestsellers

In [19]:
#%pip install selenium
#%pip install undetected-chromedriver

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import undetected_chromedriver as uc
import random
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
import numpy as np


In [20]:
driver = uc.Chrome(use_subprocess=True) #using undetected chromedriver
time.sleep(random.uniform(1,2.5))
driver.get(url)
time.sleep(random.uniform(4,5))
page_text = driver.page_source
soup = BeautifulSoup(page_text,'html.parser')
all_reviews = soup.find('span', {"class":"cr-widget-FocalReviews"}).find('div', {"class":"a-row"}).find('div', {"data-hook":"top-customer-reviews-widget"})
top_reviews = all_reviews.find_all('div', {"data-hook":"review"})
driver.quit()

In [21]:
review_list = []
for i in top_reviews:
    score = i.find('a', {"data-hook":"review-title"}).find('i', {"data-hook":"review-star-rating"}).text
    review = i.find('div', {"class":"a-row a-spacing-small review-data"}).text
    review_list.append([review, score])

In [22]:
df_live_reviews = pd.DataFrame(review_list, columns=["review", "score"])
# Clean the "score" column
df_live_reviews['score'] = df_live_reviews['score'].str.extract(r'(\d+)\.0 out')
df_live_reviews['sentiment'] = np.where(df_live_reviews['score'].astype(float) >= 4, 'Positive', 'Negative')

df_live_reviews = df_live_reviews[['review', 'sentiment']]
df_live_reviews

,review,sentiment
0,\nMy Nord vpn license was due to expire so alr...,Positive
1,\nEasy to set up and use. Great product.\nRead...,Positive
2,\nI wish I knew my router was not covered by N...,Negative
3,\nExcellent\nRead more,Positive
4,\nEasy to use great customer support highly re...,Positive
5,\nI bought this product last year and it was g...,Positive
6,\nUnable to get to work\nRead more,Negative
7,\nthe activation code for nordvpn is not worki...,Negative


Preparing the data!

In [23]:
import pandas as pd
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot, Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Activation, Dropout, Dense
#from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, GlobalMaxPooling1D, Embedding, Conv1D, LSTM
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\karou\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [24]:
import re

TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):

    return TAG_RE.sub('', text)

In [25]:
from nltk.corpus import stopwords

def preprocess_text(sen):

    sentence = sen.lower()

    sentence = remove_tags(sentence)

    #removing punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)

    #removing single characters
    sentence = re.sub(r"\s+[^a-zA-Z]\s+", ' ', sentence)

    #removing multiple space
    sentence = re.sub(r'\s+', ' ', sentence)

    #remove stopwords
    pattern = re.compile(r'\b(' + r'|'.join(stopwords.words('english')) + r')\b\s*')
    sentence = pattern.sub('', sentence)

    return sentence

In [26]:
from tqdm import tqdm

X = []

sentences = list(df_live_reviews['review'])

for sen in tqdm(sentences):
    X.append(preprocess_text(sen))

100%|██████████| 8/8 [00:00<00:00, 888.79it/s]


In [27]:
X[0]

' nord vpn license due expire already quote nord checked amazon surprise much cheaper bought license received email strait away added license key account updated renewal date notice renewed nord give option cancel subscription trying auto renew using expired credit card would recommend anyone thinking using nord buy amazon supply credit card details read '

In [28]:
word_tokenizer = Tokenizer()
word_tokenizer.fit_on_texts(X)
sequences = word_tokenizer.texts_to_sequences(X)
max_sequence_length = 100  # The same value used during training
X_test = pad_sequences(sequences, maxlen=max_sequence_length)

In [29]:
lstm_model = load_model('c1_lstm_model_acc_0.869.h5')

predictions = lstm_model.predict(X_test)


1/1 [==============================] - 2s 2s/step


In [34]:
# Convert predicted probabilities to sentiment labels
predicted_sentiments = ['Positive' if pred > 0.6 else 'Negative' for pred in predictions]

# Add predicted sentiments to the dataframe
df_live_reviews['predicted_sentiment'] = predicted_sentiments
df_live_reviews['predicted_prob'] = predictions

In [35]:
df_live_reviews

,review,sentiment,predicted_sentiment,predicted_prob
0,\nMy Nord vpn license was due to expire so alr...,Positive,Positive,0.836820
1,\nEasy to set up and use. Great product.\nRead...,Positive,Positive,0.652570
2,\nI wish I knew my router was not covered by N...,Negative,Positive,0.802692
3,\nExcellent\nRead more,Positive,Positive,0.786988
4,\nEasy to use great customer support highly re...,Positive,Negative,0.418971
5,\nI bought this product last year and it was g...,Positive,Positive,0.726164
6,\nUnable to get to work\nRead more,Negative,Positive,0.638656
7,\nthe activation code for nordvpn is not worki...,Negative,Positive,0.737987


In [37]:
X[6]

' unable get work read '